## Autoencoder with single hidden layer

In [1]:
import tensorflow as tf
import numpy as np
from sklearn import datasets

In [2]:
def get_batch(X,size):
    return X[np.random.choice(len(X), size, replace = False)]

class Autoencoder:
    def __init__(self, input_dim, hidden_dim, batch_size = 10, epoch=250, learning_rate=0.001):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.epoch = epoch
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        
        x = tf.placeholder(tf.float32,[None, input_dim])    # N_samples x N_features
        
        with tf.name_scope('encoder'):
            weights = tf.Variable(tf.random_normal([input_dim, hidden_dim]), dtype = tf.float32, name='weights') # N_features x N_hidden
            bias = tf.Variable(tf.zeros([1,hidden_dim]), dtype = tf.float32, name = 'bias') # 1 x N_hidden
            encoded = tf.nn.tanh(x @ weights + bias) # N_samples x N_hidden
        with tf.name_scope('decoder'):
            weights = tf.Variable(tf.random_normal([ hidden_dim, input_dim]), dtype = tf.float32, name='weights') # N_hiddenx N_features
            bias = tf.Variable(tf.zeros([1,input_dim]), dtype = tf.float32, name = 'bias')  # 1 x N_features
            decoded = (encoded @ weights + bias) # N_samples x N_features
        
        self.x = x
        self.encoded = encoded
        self.decoded = decoded
            
        self.cost = tf.sqrt(tf.reduce_mean(tf.square(self.x-self.decoded)))
        self.train_op = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.cost)
        # Gradient descent is slowest: Here are other choices -
        # tf.train.AdagradOptimizer, tf.train.AdagradDAOptimizer,
        # tf.train.MomentumOptimizer, tf.train.AdamOptimizer, tf.train.FtrlOptimizer
        self.saver = tf.train.Saver()
        
    def train(self, data):
        num_samples = len(data)
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for i in range(self.epoch):
                for j in range(500):
                    batch_data = get_batch(data, self.batch_size)
                    loss, _ = sess.run([self.cost, self.train_op], feed_dict={self.x : batch_data})
                if i % 10 ==0: 
                    print('epoch {} : loss = {}' .format(i,loss))
                    self.saver.save(sess,'./model.ckpt')
            self.saver.save(sess,'./model.ckpt')
        
    def test(self, data):
        with tf.Session() as sess:
            self.saver.restore(sess,'./model.ckpt')
            hidden, reconstructed = sess.run([self.encoded, self.decoded], feed_dict = {self.x : data})
            print('Input : ', data)
            print('Compressed : ', hidden)
            print('Reconstructed : ', reconstructed)
        return reconstructed
                    
                    
                               
        

In [3]:
hidden_dim = 2
data = datasets.load_iris().data

In [4]:
input_dim = len(data[0])
ae = Autoencoder(input_dim, hidden_dim)
ae.train(data)
ae.test([[8, 4, 6, 2]])

Instructions for updating:
Colocations handled automatically by placer.
epoch 0 : loss = 3.434065103530884
epoch 10 : loss = 0.3701309263706207
epoch 20 : loss = 0.23624461889266968
epoch 30 : loss = 0.23182491958141327
epoch 40 : loss = 0.14751219749450684
epoch 50 : loss = 0.16832223534584045
epoch 60 : loss = 0.1880117505788803
epoch 70 : loss = 0.17619356513023376
epoch 80 : loss = 0.15801282227039337
epoch 90 : loss = 0.17461101710796356
epoch 100 : loss = 0.18727660179138184
epoch 110 : loss = 0.11468915641307831
epoch 120 : loss = 0.163792222738266
epoch 130 : loss = 0.18811912834644318
epoch 140 : loss = 0.16835914552211761
epoch 150 : loss = 0.2223578244447708
epoch 160 : loss = 0.18948659300804138
epoch 170 : loss = 0.18121179938316345
epoch 180 : loss = 0.12824703752994537
epoch 190 : loss = 0.1723240166902542
epoch 200 : loss = 0.15339426696300507
epoch 210 : loss = 0.16876456141471863
epoch 220 : loss = 0.19350369274616241
epoch 230 : loss = 0.14975221455097198
epoch 240 :

array([[7.7336464, 3.8457012, 5.8278704, 2.0732484]], dtype=float32)

## Images

In [5]:
import pickle

def unpickle(file):
    fo = open(file, 'rb')
    dict = pickle.load(fo, encoding='latin1')
    fo.close()
    return dict

names = unpickle('./cifar-10-batches-py/batches.meta')['label_names']
data, labels = [], []
for i in range(1, 6):
    filename = './cifar-10-batches-py/data_batch_' + str(i)
    batch_data = unpickle(filename)
    if len(data) > 0:
        data = np.vstack((data, batch_data['data']))
        labels = np.hstack((labels, batch_data['labels']))
    else:
        data = batch_data['data']
        labels = batch_data['labels']
        
def grayscale(a):
    return a.reshape(a.shape[0], 3, 32, 32).mean(1).reshape(a.shape[0], -1)

data = grayscale(data)

In [6]:
x = np.matrix(data)
y = np.array(labels)
horse_indices = np.where(y == 7)[0]
horse_x = x[horse_indices]
print(np.shape(horse_x)) # (5000, 3072)
input_dim = np.shape(horse_x)[1]
hidden_dim = 100
ae = Autoencoder(input_dim, hidden_dim,epoch=200)
ae.train(horse_x)

(5000, 1024)
epoch 0 : loss = 94.04011535644531
epoch 10 : loss = 52.74992752075195
epoch 20 : loss = 54.91603469848633
epoch 30 : loss = 55.78679656982422
epoch 40 : loss = 49.97771072387695
epoch 50 : loss = 57.22572326660156
epoch 60 : loss = 46.68358612060547
epoch 70 : loss = 47.60668182373047
epoch 80 : loss = 54.73023986816406
epoch 90 : loss = 43.55459213256836
epoch 100 : loss = 49.520469665527344
epoch 110 : loss = 48.13953399658203
epoch 120 : loss = 47.695072174072266
epoch 130 : loss = 45.13964080810547
epoch 140 : loss = 48.41259002685547
epoch 150 : loss = 46.515403747558594
epoch 160 : loss = 48.10332107543945
epoch 170 : loss = 47.239131927490234
epoch 180 : loss = 47.88938522338867
epoch 190 : loss = 53.45363235473633


In [7]:
data[np.newaxis,512,:]

array([[165.33333333, 154.33333333, 163.33333333, ...,  83.33333333,
         81.66666667,  79.33333333]])

In [8]:
ae.test(data[np.newaxis,512,:])

INFO:tensorflow:Restoring parameters from ./model.ckpt
Input :  [[165.33333333 154.33333333 163.33333333 ...  83.33333333  81.66666667
   79.33333333]]
Compressed :  [[ 1.  1. -1. -1.  1. -1.  1. -1.  1. -1.  1.  1.  1. -1. -1.  1. -1. -1.
  -1.  1.  1. -1. -1. -1.  1.  1. -1. -1. -1.  1. -1.  1.  1.  1.  1. -1.
  -1. -1. -1. -1. -1.  1.  1.  1. -1.  1.  1. -1. -1. -1. -1.  1. -1.  1.
  -1. -1. -1. -1.  1.  1.  1. -1.  1.  1. -1. -1.  1.  1. -1.  1. -1.  1.
  -1. -1.  1. -1.  1. -1.  1.  1. -1. -1. -1. -1. -1.  1.  1. -1.  1. -1.
   1. -1. -1. -1.  1. -1. -1. -1. -1.  1.]]
Reconstructed :  [[150.61594  150.97699  152.04547  ...  70.15095   72.889755  72.77891 ]]


array([[150.61594 , 150.97699 , 152.04547 , ...,  70.15095 ,  72.889755,
         72.77891 ]], dtype=float32)